In [1]:
from glob import glob
import json

In [2]:
files = glob('output-malaysia-location/*.json')
len(files)

13464

In [33]:
with open(files[100]) as fopen:
    d = json.load(fopen)

In [34]:
d[0]['filename']

'/home/ubuntu/.cache/huggingface/hub/datasets--malaysia-ai--crawl-google-image-malaysia-location/snapshots/da8d9dc03a7d6779de6062836030a1c90c0e74ea/data/train-00594-of-01000.parquet'

In [35]:
d[100]['filename']

'/home/ubuntu/.cache/huggingface/hub/datasets--malaysia-ai--crawl-google-image-malaysia-location/snapshots/da8d9dc03a7d6779de6062836030a1c90c0e74ea/data/train-00594-of-01000.parquet'

In [5]:
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import streaming
import numpy as np
from tqdm import tqdm
import os

class Float32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.float32)

_encodings['float32'] = Float32

columns = {
    'embedding': 'float32',
    'label': 'str',
    'filename': 'str',
    'index': 'int'
}
hashes = 'sha1', 'xxh64'

In [6]:
!mkdir embedding

mkdir: cannot create directory ‘embedding’: File exists


In [36]:
def loop(files):
    files, index = files
    out_root = f'embedding/embedding-{index}'
    os.system(f'rm -rf {out_root}')
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for f in tqdm(files):
            with open(f) as fopen:
                data = json.load(fopen)
                for row in data:
                    row['embedding'] = np.array(row['embedding'], dtype = np.float32)
                    out.write(row)

In [37]:
loop((files[:2], 0))

100%|██████████| 2/2 [00:00<00:00, 27.30it/s]


In [38]:
import mp

mp.multiprocessing(files, loop, cores = min(len(files), 30), returned = False)

100%|██████████| 24/24 [00:00<00:00, 28.45it/s]


In [40]:
folders = sorted(glob('embedding/embedding-*'), key = lambda x: int(x.split('-')[-1]))
len(folders)

31

In [41]:
!rm -rf combine

In [42]:
with MDSWriter(out='combine', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|██████████| 7200/7200 [00:00<00:00, 24252.13it/s]


In [46]:
dataset = LocalDataset('combine')

In [47]:
len(dataset)

4037127

In [48]:
dataset[100000]['filename']

'/home/ubuntu/.cache/huggingface/hub/datasets--malaysia-ai--crawl-google-image-malaysia-location/snapshots/da8d9dc03a7d6779de6062836030a1c90c0e74ea/data/train-00990-of-01000.parquet'

In [49]:
mapping = {}
for i in tqdm(range(len(dataset))):
    f = os.path.split(dataset[i]['filename'])[1]
    key = f"{f}-{dataset[i]['index']}"
    mapping[key] = i

100%|██████████| 4037127/4037127 [04:08<00:00, 16252.34it/s]


In [50]:
len(mapping)

4036638

In [51]:
with open('mapping-index.json', 'w') as fopen:
    json.dump(mapping, fopen)

In [52]:
from huggingface_hub import HfApi
api = HfApi()

In [53]:
api.upload_folder(
    folder_path='combine',
    repo_id='mesolitica/google-image-malaysia-location-embedding',
    repo_type='dataset',
)

Upload 198 LFS files:   0%|          | 0/198 [00:00<?, ?it/s]

shard.00002.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00003.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00000.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00001.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

shard.00194.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00195.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00196.mds:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

shard.00197.mds:   0%|          | 0.00/40.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/google-image-malaysia-location-embedding/commit/d922a45ca33747d2f875707e4311bc9f3cf7644f', commit_message='Upload folder using huggingface_hub', commit_description='', oid='d922a45ca33747d2f875707e4311bc9f3cf7644f', pr_url=None, pr_revision=None, pr_num=None)

In [55]:
api.upload_file(
    path_or_fileobj='mapping-index.json',
    path_in_repo='mapping-index.json',
    repo_id='mesolitica/google-image-malaysia-location-embedding',
    repo_type='dataset',
)

mapping-index.json:   0%|          | 0.00/183M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/google-image-malaysia-location-embedding/commit/d40d4889c5f609295de99387b1c9ec22217217a2', commit_message='Upload mapping-index.json with huggingface_hub', commit_description='', oid='d40d4889c5f609295de99387b1c9ec22217217a2', pr_url=None, pr_revision=None, pr_num=None)